In [1]:
from transformers import pipeline
import pandas as pd
from ipywidgets import widgets

In [2]:
qa_model = pipeline(task='table-question-answering',
                  model='dsba-lab/koreapas-finetuned-korwikitq')

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
# 답변용 데이터 불러오기
df = pd.read_csv('./python_exception.csv')
df.head()

,Error,설명,Unnamed: 2,예제
0,AssertionError,assert 문이 실패할 때 발생하는 에러입니다. assert문은 주어진 조건이 거...,NaN,">>> x = 999\n>>> assert x != 999, 'x의 값이 예상과 다..."
1,AttributeError,"대상이 지원하지 않는 함수를 사용하려 할 때 발생하는 에러입니다. 예를 들어, 숫자...",NaN,>>> x = 9989\n>>> x.split(8)\n\nAttributeError...
2,EOFError,input() 함수가 데이터를 읽지 못한 상태에서 EOF (end-of-file) ...,NaN,NaN
3,FloatingPointError,현재 사용되지 않습니다.,NaN,NaN
4,GeneratorExit,제너레이터 또는 코루틴 이 닫힐 때 발생합니다,generator.close() 와 coroutine.close() 를 보십시오....,NaN


In [4]:
df = df.iloc[:,:-2].copy()

In [5]:
qa_model(query='zerodivison이 뭐야?',table=df)

/opt/anaconda3/lib/python3.12/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/lib/python3.12/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


{'answer': '나누기 또는 모듈로 연산의 두 번째 인자가 0일 때 발생합니다. 연관된 값은 피연산자의 형과 연산을 나타내는 문자열입니다.',
 'coordinates': [(32, 1)],
 'cells': ['나누기 또는 모듈로 연산의 두 번째 인자가 0일 때 발생합니다. 연관된 값은 피연산자의 형과 연산을 나타내는 문자열입니다.']}

In [6]:
qa_model(query='assert 문이 실패할 때 발생하는 에러입니다',table=df)

/opt/anaconda3/lib/python3.12/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/lib/python3.12/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


{'answer': 'AssertionError',
 'coordinates': [(0, 0)],
 'cells': ['AssertionError']}

In [16]:
def make_answer(q):
    # QA 모델을 사용해 질문에 대한 답을 받음
    answer = qa_model(query=q, table=df)
    
    # 답이 빈 문자열인 경우
    if answer['answer'] == '':
        return '질문에 대한 답을 찾지 못했습니다.'
    
    # 좌표가 유효하고, answer['coordinates']가 비어있지 않은지 확인
    elif 'coordinates' in answer and answer['coordinates'] and answer['coordinates'][0][-1] == 0:
        answer2 = answer['answer']
        설명_row = df[df['Error'] == answer2]

        if not 설명_row.empty:
            설명_text = 설명_row['설명'].iloc[0]
            return f'''질문이나 코드에서 파악한 에러의 이름은 {answer2} 입니다.
    에러가 발생하는 원인은 다음과 같습니다: {설명_text}'''
        else:
            return '에러에 대한 설명을 찾을 수 없습니다.'
    
    # 'cells'에 접근 가능한지 확인
    elif 'cells' in answer and answer['cells']:
        answer2 = answer['cells'][0]
        error_row = df[df['설명'] == answer2]

        if not error_row.empty:
            error_name = error_row['Error'].iloc[0]
            return f'''
                질문이나 코드에서 파악된 문제는 {answer2}
                이 에러의 이름은 {error_name} 입니다.
                '''
        else:
            return '문제에 대한 에러 이름을 찾을 수 없습니다.'
    
    # 위의 조건에 모두 해당하지 않을 경우
    return '질문에 대한 명확한 답변을 찾지 못했습니다.'


In [22]:
inputs = input('질문이나 에러 메시지를 입력해주세요. : ')
# inputs = Textarea('질문이나 에러 메시지를 입력해주세요. : ')

return_msg = make_answer(inputs)
print(return_msg)

/opt/anaconda3/lib/python3.12/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/lib/python3.12/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]



                질문이나 코드에서 파악된 문제는 나누기 또는 모듈로 연산의 두 번째 인자가 0일 때 발생합니다. 연관된 값은 피연산자의 형과 연산을 나타내는 문자열입니다.
                이 에러의 이름은 ZeroDivisionError 입니다.
                


In [35]:
data = {
    '모델 이름': [
        'ResNet', 'BERT', 'XGBoost', 'LSTM', 'K-Means', 'Random Forest', 'Transformer', 'CNN', 'SVM',
        'LightGBM', 'Naive Bayes', 'GAN', 'Autoencoder', 'VGG', 'Decision Tree', 'RNN', 'MLP'
    ],
    '주요 용도': [
        '이미지 분류', '자연어 처리', '회귀 및 분류', '시계열 예측', '클러스터링', '회귀 및 분류', '자연어 처리', '이미지 처리', '분류',
        '회귀 및 분류', '분류', '생성 모델', '비지도 학습', '이미지 분류', '회귀 및 분류', '시계열 예측', '회귀 및 분류'
    ],
    '데이터 유형': [
        '이미지', '텍스트', '탭 데이터', '시계열 데이터', '탭 데이터', '탭 데이터', '텍스트', '이미지', '탭 데이터',
        '탭 데이터', '탭 데이터', '이미지 및 텍스트', '이미지 및 텍스트', '이미지', '탭 데이터', '시계열 데이터', '탭 데이터'
    ],
    '모델 특징': [
        'Residual connections을 사용한 깊은 네트워크',
        'Transformer 기반 언어 모델, 문장 이해에 탁월',
        '의사결정 나무 기반 부스팅 모델, 성능 우수',
        '순환 신경망 기반, 시계열 데이터에 강점',
        '비지도 학습 기반 클러스터링 알고리즘',
        '다수의 결정 트리를 앙상블로 결합',
        '자연어 처리에서 널리 사용되는 트랜스포머 구조',
        '이미지 처리에 적합한 합성곱 신경망',
        '고차원 공간에서의 분류 문제 해결',
        'Gradient Boosting 방식으로 빠르고 효율적',
        '조건부 독립 가정을 사용하는 확률 모델',
        'Generator와 Discriminator로 데이터 생성',
        '입력 데이터의 압축 및 재구성',
        '16개의 합성곱 계층을 가진 깊은 신경망',
        '단순하지만 해석 가능성이 높은 모델',
        '순환 신경망, 시계열 데이터 처리',
        '여러 층의 완전 연결 신경망, 일반적인 태스크에 사용'
    ],
    '주요 알고리즘': [
        'Convolutional Neural Networks', 'Transformer', 'Gradient Boosting', 'Recurrent Neural Networks',
        'K-means Clustering', 'Ensemble Learning', 'Attention Mechanism', 'Convolutional Neural Networks',
        'Support Vector Machine', 'Gradient Boosting', 'Bayesian Probability', 'Generative Adversarial Network',
        'Autoencoder', 'Convolutional Neural Networks', 'Decision Tree', 'Recurrent Neural Networks',
        'Multi-Layer Perceptron'
    ]
}

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   모델 이름    17 non-null     object
 1   주요 용도    17 non-null     object
 2   데이터 유형   17 non-null     object
 3   모델 특징    17 non-null     object
 4   주요 알고리즘  17 non-null     object
dtypes: object(5)
memory usage: 812.0+ bytes


In [41]:
df = pd.DataFrame(data)
df.head()

,모델 이름,주요 용도,데이터 유형,모델 특징,주요 알고리즘
0,ResNet,이미지 분류,이미지,Residual connections을 사용한 깊은 네트워크,Convolutional Neural Networks
1,BERT,자연어 처리,텍스트,"Transformer 기반 언어 모델, 문장 이해에 탁월",Transformer
2,XGBoost,회귀 및 분류,탭 데이터,"의사결정 나무 기반 부스팅 모델, 성능 우수",Gradient Boosting
3,LSTM,시계열 예측,시계열 데이터,"순환 신경망 기반, 시계열 데이터에 강점",Recurrent Neural Networks
4,K-Means,클러스터링,탭 데이터,비지도 학습 기반 클러스터링 알고리즘,K-means Clustering


In [42]:
question_input = widgets.Text(
    value='',
    placeholder='질문을 입력하세요',
    description='질문:',
    disabled=False
)

submit_button = widgets.Button(
    description='질문하기',
    disabled=False,
    button_style='info',
    tooltip='질문을 제출하고 결과를 확인합니다.',
    icon='check'
)

output_area = widgets.Output()

In [43]:
def on_submit_question(b):
    with output_area:
        output_area.clear_output(wait=True)  # 이전 출력 내용 삭제 (clear_output에 wait=True 추가)
        question = question_input.value
        if question:
            try:
                # 모델을 사용하여 질문에 대한 답을 생성
                answer = qa_model(table=df, query=question)
                
                # 답변이 비어있지 않다면 출력
                if answer and 'answer' in answer and answer['answer']:
                    print(f"질문: {question}")
                    print(f"답변: {answer['answer']}")
                else:
                    print(f"질문: {question}")
                    print("답변을 찾을 수 없습니다.")
            except Exception as e:
                # 예외 발생 시 출력
                print(f"오류 발생: {str(e)}")
                print("질문에 대한 답변을 생성하는 도중 오류가 발생했습니다.")
        else:
            print("질문을 입력해주세요.")

In [44]:
# 버튼 클릭 이벤트 연결
submit_button.on_click(on_submit_question)

# GUI 구성 표시
display(question_input, submit_button, output_area)

Text(value='', description='질문:', placeholder='질문을 입력하세요')

Button(button_style='info', description='질문하기', icon='check', style=ButtonStyle(), tooltip='질문을 제출하고 결과를 확인합니다…

Output()

In [97]:
weather = pd.read_csv('extremum_20240911111909.csv',encoding='euc-kr')
weather

,\t\t지점번호,지점명,일시,평균기온(℃),최고기온(℃),\t최고기온시각,최저기온(℃),최저기온시각일교차,Unnamed: 8
0,\t\t108,서울,2020.1.1,-2.2,0.3,14:57,-6.5,0:01,6.8
1,\t\t108,서울,2020.1.2,1.0,3.8,15:00,-0.7,0:01,4.5
2,\t\t108,서울,2020.1.3,-0.1,4.6,15:47,-3.4,8:07,8.0
3,\t\t108,서울,2020.1.4,1.2,6.1,14:50,-2.8,5:12,8.9
4,\t\t108,서울,2020.1.5,1.3,6.6,14:53,-3.2,8:21,9.8
...,...,...,...,...,...,...,...,...,...
1718,\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1719,\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1720,\t\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1721,\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
weather = weather.dropna()

In [99]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1714 entries, 0 to 1714
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   		지점번호      1714 non-null   object 
 1   지점명         1714 non-null   object 
 2   일시          1714 non-null   object 
 3   평균기온(℃)     1714 non-null   float64
 4   최고기온(℃)     1714 non-null   float64
 5   	최고기온시각     1714 non-null   object 
 6   최저기온(℃)     1714 non-null   float64
 7   최저기온시각일교차   1714 non-null   object 
 8   Unnamed: 8  1714 non-null   float64
dtypes: float64(4), object(5)
memory usage: 133.9+ KB


In [100]:
weather.rename(columns={'Unnamed: 8': '일교차'}, inplace=True)

/var/folders/dn/kpsvhr056zz6gvcnmgrkklzr0000gn/T/ipykernel_45203/3250655369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather.rename(columns={'Unnamed: 8': '일교차'}, inplace=True)


In [101]:
weather

,\t\t지점번호,지점명,일시,평균기온(℃),최고기온(℃),\t최고기온시각,최저기온(℃),최저기온시각일교차,일교차
0,\t\t108,서울,2020.1.1,-2.2,0.3,14:57,-6.5,0:01,6.8
1,\t\t108,서울,2020.1.2,1.0,3.8,15:00,-0.7,0:01,4.5
2,\t\t108,서울,2020.1.3,-0.1,4.6,15:47,-3.4,8:07,8.0
3,\t\t108,서울,2020.1.4,1.2,6.1,14:50,-2.8,5:12,8.9
4,\t\t108,서울,2020.1.5,1.3,6.6,14:53,-3.2,8:21,9.8
...,...,...,...,...,...,...,...,...,...
1710,\t\t108,서울,2024.9.6,24.0,25.7,0:01,23.3,12:11,2.4
1711,\t\t108,서울,2024.9.7,26.5,31.3,14:32,22.8,6:21,8.5
1712,\t\t108,서울,2024.9.8,27.2,32.8,15:28,22.6,6:35,10.2
1713,\t\t108,서울,2024.9.9,28.5,34.1,14:45,24.1,4:20,10.0


In [102]:
weather.columns = weather.columns.str.replace('\t','')
weather

,지점번호,지점명,일시,평균기온(℃),최고기온(℃),최고기온시각,최저기온(℃),최저기온시각일교차,일교차
0,\t\t108,서울,2020.1.1,-2.2,0.3,14:57,-6.5,0:01,6.8
1,\t\t108,서울,2020.1.2,1.0,3.8,15:00,-0.7,0:01,4.5
2,\t\t108,서울,2020.1.3,-0.1,4.6,15:47,-3.4,8:07,8.0
3,\t\t108,서울,2020.1.4,1.2,6.1,14:50,-2.8,5:12,8.9
4,\t\t108,서울,2020.1.5,1.3,6.6,14:53,-3.2,8:21,9.8
...,...,...,...,...,...,...,...,...,...
1710,\t\t108,서울,2024.9.6,24.0,25.7,0:01,23.3,12:11,2.4
1711,\t\t108,서울,2024.9.7,26.5,31.3,14:32,22.8,6:21,8.5
1712,\t\t108,서울,2024.9.8,27.2,32.8,15:28,22.6,6:35,10.2
1713,\t\t108,서울,2024.9.9,28.5,34.1,14:45,24.1,4:20,10.0


In [103]:
weather = weather.iloc[:,2:]
weather

,일시,평균기온(℃),최고기온(℃),최고기온시각,최저기온(℃),최저기온시각일교차,일교차
0,2020.1.1,-2.2,0.3,14:57,-6.5,0:01,6.8
1,2020.1.2,1.0,3.8,15:00,-0.7,0:01,4.5
2,2020.1.3,-0.1,4.6,15:47,-3.4,8:07,8.0
3,2020.1.4,1.2,6.1,14:50,-2.8,5:12,8.9
4,2020.1.5,1.3,6.6,14:53,-3.2,8:21,9.8
...,...,...,...,...,...,...,...
1710,2024.9.6,24.0,25.7,0:01,23.3,12:11,2.4
1711,2024.9.7,26.5,31.3,14:32,22.8,6:21,8.5
1712,2024.9.8,27.2,32.8,15:28,22.6,6:35,10.2
1713,2024.9.9,28.5,34.1,14:45,24.1,4:20,10.0


In [ ]:
question_input = widgets.Text(
    value='',
    placeholder='질문을 입력하세요',
    description='질문:',
    disabled=False
)

submit_button = widgets.Button(
    description='질문하기',
    disabled=False,
    button_style='info',
    tooltip='질문을 제출하고 결과를 확인합니다.',
    icon='check'
)

output_area = widgets.Output()